In [16]:
import os
import re
import pymupdf
import django_setup
from IPython.display import Markdown, display
from bs4 import BeautifulSoup as bs

from django.conf import settings

from cmj.utils import clean_text

gemini_api_key = settings.GEMINI_API_KEY

In [17]:
import google.generativeai as genai
genai.configure(api_key=gemini_api_key)


In [18]:
def get_model_configured():
    generation_config = {
      "temperature": 0.1,
      "top_p": 0.95,
      "top_k": 40,
      #"max_output_tokens": 8192,
      "response_mime_type": "text/plain",
    }

    model = genai.GenerativeModel(
      model_name="gemini-2.0-flash-exp",
      generation_config=generation_config,
    )

    return model

model = get_model_configured()


In [81]:
def make_prompt(original, analisado, o_epigrafe, a_epigrafe):


    prompt = f"""    Assuma a personalidade de um especialista em Gestão Pública Municipal com mais de 10 anos de conhecimento teórico e atividade prática. Você é um agente do poder executivo municipal que está recebendo dois requerimentos de vereadores. Seu papel é comparar o conteúdo de <ORIGINAL></ORIGINAL> com o conteúdo de <ANALISADO></ANALISADO>. Para citar estes dois conteúdos, nomeie eles respectivamente da seguinte maneira: "{o_epigrafe}" e "{a_epigrafe}". Remova de sua análise os autores e os destinatários pois são irrelevantes para a comparação requerida. O importante é o que está sendo pedido, quem será o beneficiário do pedido e para qual localidade está sendo feito tal pedido. Não faça considerações adicionais e ou mesmo conclusão extra. Neste contexto responda:

    - Os textos estão pedindo o mesmo benefício para a mesma localidade? Descreva de forma sucinta e direta o que está sendo pedido em <ORIGINAL></ORIGINAL> e <ANALISADO></ANALISADO> informando também os beneficiários e as localidades citadas.
    - Calcule a semelhança percentual entre Desconside autores e destinatários, focando na solicitação e no beneficiário, qual semelhança percentual entre <ORIGINAL></ORIGINAL> e <ANALISADO></ANALISADO>?


<ORIGINAL>{original}</ORIGINAL>

<ANALISADO>{analisado}</ANALISADO>
    """

    prompt2 = f"""    Compare o conteúdo de <ORIGINAL></ORIGINAL>, ao qual será chamado daqui em diante de "T.O.", com o conteúdo de <ANALISADO></ANALISADO>, ao qual será chamado daqui em diante de "T.A.", e escreva em formato MARKDOWN respondendo as perguntas numeradas que estão em <PERGUNTAS></PERGUNTAS>.

<PERGUNTAS>
    1) "T.O." e "T.A." fazem solicitações a seus destinatários? Quais são essas solicitações? E qual o beneficiário de cada uma das solicitações?
    2) Desconsiderando autores e destinatários, focando na solicitação e no beneficiário, qual grau de semelhança, de zero a um, poderia ser atribuído entre "T.O." e "T.A."?
</PERGUNTAS>

<ORIGINAL>{original}</ORIGINAL>

<ANALISADO>{analisado}</ANALISADO>
    """

    prompt1 = f"""    Assuma a personalidade de um especialista em plágio de documentos. É muito importante que em sua resposta nunca seja usada a palavra "plágio", em vez disso, se necessário, utilize a palavra similaridade.
    Compare o conteúdo de <ORIGINAL></ORIGINAL>, ao qual será chamado daqui em diante de "T.O.", com o conteúdo de <ANALISADO></ANALISADO>, ao qual será chamado daqui em diante de "T.A." e escreva em formato MARKDOWN respondendo as perguntas numeradas que estão em <PERGUNTAS></PERGUNTAS>. Seja Conciso.

<PERGUNTAS>
    1) Existe semelhança temática entre "T.O." e "T.A."? Quais são estas semelhanças? É o mesmo tema (frize)?
    2) Existe semelhança nas solicitações feita em "T.O." e "T.A."? Quais são estas semelhanças?
    3) A solicitações feitas é uma melhoria para algum lugar, seja esse lugar um logradouro público, um bairro, um prédio comercial, um prédio público, ou ainda instituições públicas e/ou privadas. Que lugar é este? É o mesmo lugar?
    4) Alguma consideração a fazer quanto a similaridade entre "T.O." e "T.A."?
    5) Desconsiderando autores e destinatários, focando na solicitação e no beneficiário, qual grau de semelhança, de zero a um, poderia ser atribuído entre "T.O." e "T.A."?
</PERGUNTAS>

<ORIGINAL>{original}</ORIGINAL>

<ANALISADO>{analisado}</ANALISADO>
    """

    while '  ' in prompt:
        prompt = prompt.replace('  ', ' ')

    return prompt


In [86]:
from sapl.materia.models import MateriaLegislativa

originais = MateriaLegislativa.objects.filter(numero=45, tipo_id=3, ano=2025).order_by('id')
analisadas = MateriaLegislativa.objects.filter(numero=46, tipo_id=3, ano=2025).order_by('id')

fez_um_doc = False
for original in originais:
    for analisada in analisadas:
        if original == analisada:
            continue

        if not set(original.autores.all()) - set(analisada.autores.all()):
            continue

        fez_um_doc = True

        print(original, '≃', analisada)

        doc_original = pymupdf.open(original.texto_original.path)
        text_original = ' '.join([page.get_text() for page in doc_original])
        text_original = clean_text(text_original)

        doc_analisada = pymupdf.open(analisada.texto_original.path)
        text_analisada = ' '.join([page.get_text() for page in doc_analisada])
        text_analisada = clean_text(text_analisada)

        prompt = make_prompt(text_original, text_analisada, original.epigrafe_short, analisada.epigrafe_short)

        answer = model.generate_content(prompt)
        display(Markdown(answer.text))

        if fez_um_doc:
            break

    if fez_um_doc:
        break


Requerimento nº 45 de 2025 ≃ Requerimento nº 46 de 2025


- Os textos não pedem o mesmo benefício para a mesma localidade.

   - **REQ 045/2025:** Solicita a ampliação do atendimento em turno integral para a educação infantil de 0 a 5 anos em todas as instituições públicas de educação infantil (CEMEIs/CEIs) do município de Jataí. Os beneficiários são as crianças de 0 a 5 anos matriculadas nas instituições públicas de educação infantil de Jataí.

   - **REQ 046/2025:** Solicita um estudo de viabilidade técnica para a construção de uma escola em tempo integral no município de Jataí. Os beneficiários são os futuros alunos da escola em tempo integral a ser construída no município de Jataí.

- A semelhança percentual entre os requerimentos é de 0%. Embora ambos tratem de educação em tempo integral no município de Jataí, um solicita a ampliação do atendimento em instituições já existentes e o outro solicita um estudo para construção de uma nova escola.
